In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tqdm import tqdm
#retrieve links
import requests
import warnings
warnings.filterwarnings("ignore")
import time
from pathlib import Path

In [2]:
folder = Path("")
files = (folder.glob("*.tfrecord"))

In [3]:
filenames=[]
for file in files:
    filenames.append(str(file))

In [4]:
raw_dataset = tf.data.TFRecordDataset(filenames)

2022-07-15 10:37:48.974582: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-07-15 10:37:48.974947: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-15 10:37:48.976547: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [5]:
feature_description = {
    'id': tf.io.FixedLenFeature([], tf.string, default_value='')
}

In [6]:
def _parse_function(example_proto):
  # Parse the input `tf.train.Example` proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, feature_description)


parsed_dataset = raw_dataset.map(_parse_function)
parsed_dataset

<MapDataset shapes: {id: ()}, types: {id: tf.string}>

In [7]:
lista=[]
for parsed_record in parsed_dataset.take(-1):
    lista.append(repr(parsed_record['id'])[-6:-2])

2022-07-15 10:37:49.046773: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-07-15 10:37:49.047059: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2894530000 Hz


In [22]:
skippati=[]

In [23]:
f=open('links5.txt','w')
for ID in tqdm(lista[4000:]):
    url = ('https://data.yt8m.org/2/j/i/{}/{}.js').format(ID[:2],ID)
    session = requests.Session()
    session.trust_env = False
    headers = {"User-Agent": "Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36"}
    
    try:
        check = session.get(url, verify=False, stream=True, headers=headers, timeout=5)
    except requests.exceptions.RequestException as e:
        skippati.append(url)
        continue
    #if check.status_code!=200:
    #    print(url, check.status_code)
    '''
    while(check.status_code != 200): #Stop while we can't download
        check = session.get(url, verify=False, stream=True, headers=headers, timeout=5)    #update the request to connect to the page until
        print(check.status_code)                   #we manage to connect
        if check.status_code==403:
            break
    '''
    if check.status_code==403:
        skippati.append(url)
        continue
    a=(check.content)[1:-1]
    ending=(a.decode()).split(',')[1][1:-2]
    f.write(('https://www.youtube.com/watch?v={}\n').format(ending))
    #time.sleep(1)
f.close()
print(('#skipped videos: {}').format(len(skippati)))

100%|███████████████████████████████████████| 1038/1038 [38:28<00:00,  2.22s/it]

#skipped videos: 104


In [ ]:
#links1 saltati 113
#links2 saltati 114
#links3 saltati 100
#links4 saltati 97
#links5 saltati 104